In [1]:
import pandas as pd
import altair as alt
from os import walk

## Reading files

In [2]:
for _, _, f3 in walk('../Outputs/master'):
    files = f3

In [3]:
master_files = []
for file in files:
    if file[:6] == 'master':
        master_files.append(file)

In [4]:
df = pd.DataFrame()
for file in master_files:
    df_temp = pd.read_csv('../Outputs/master/'+file, encoding='latin1')
    df = df.append(df_temp, ignore_index=True)

## Exploring the data

Taking a glimpse:

In [5]:
df.head()

,Competencia,Corte,Tribunal,Page,Rol,Fecha,Caratulado,Unnamed: 7,Unnamed: 8
0,Civil,C.A. de Santiago,10º Juzgado Civil de Santiago,1,C-10839-2016,29/04/2016,BANCO DE CHILE / CASTILLO,NaN,NaN
1,Civil,C.A. de Santiago,10º Juzgado Civil de Santiago,1,C-10850-2016,29/04/2016,COOPERATIVA DEL PERSONAL DE LAUNIVERSIDAD DE C...,NaN,NaN
2,Civil,C.A. de Santiago,10º Juzgado Civil de Santiago,1,C-10852-2016,29/04/2016,ISAPRE CONSALUD S A / CISTERNAS,NaN,NaN
3,Civil,C.A. de Santiago,10º Juzgado Civil de Santiago,1,C-10855-2016,29/04/2016,MATIC-KARD S.A. / DÍAZ,NaN,NaN
4,Civil,C.A. de Santiago,10º Juzgado Civil de Santiago,1,C-10863-2016,29/04/2016,MATIC-KARD S.A. / ARANCIBIA,NaN,NaN


### Total obs:

In [6]:
print('Total observations:', len(df))

Total observations: 2130262


We have 2,130,262 cases in total.

### Column names:

In [7]:
list(df.columns)

['Competencia',
 'Corte',
 'Tribunal',
 'Page',
 'Rol',
 'Fecha',
 'Caratulado',
 'Unnamed: 7',
 'Unnamed: 8']

In [8]:
print('Total number of columns:', len(df.columns))

Total number of columns: 9


### Columns `Unnamed: 7` and `Unnamed: 8`

There are two columns with names `Unnamed: 7` and `Unnamed: 8`. We'll explore these two:

In [9]:
df['Unnamed: 7'].value_counts(dropna=False)

NaN    2130262
Name: Unnamed: 7, dtype: int64

In [10]:
df['Unnamed: 7'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Unnamed: 7, dtype: float64

In [11]:
df['Unnamed: 8'].value_counts(dropna=False)

NaN    2130262
Name: Unnamed: 8, dtype: int64

In [12]:
df['Unnamed: 8'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Unnamed: 8, dtype: float64

These columns contain only missing data. We'll drop them from this exploration.

In [13]:
df = df.drop(columns=['Unnamed: 7', 'Unnamed: 8'])

In [14]:
list(df.columns)

['Competencia', 'Corte', 'Tribunal', 'Page', 'Rol', 'Fecha', 'Caratulado']

In [15]:
print('Total number of columns:', len(df.columns))

Total number of columns: 7


### Duplicated observations

Checking duplicated rows:

In [16]:
df.duplicated(keep=False).sum()

0

We don't have duplicated rows.

Checking duplicated rows in `Tribunal` and `Rol`:

In [17]:
df[['Tribunal', 'Rol']].duplicated(keep=False).sum()

4552

4,552 observations are duplicated in the values of `Tribunal` and `Rol`. This is probably due to some cases moving to a different page when a new query of the same month/tribunal is executed. We'll exclude the column `Pages` and check duplicates again to see if that's the case indeed.

In [18]:
df.drop(columns=['Page']).duplicated(keep=False).sum()

4552

This means that we have 4,552 cases that are duplicated in all column values except `Page`. We'll check what percentage of all observations are duplicated in all columns except `Page`.

In [19]:
print('Percentage of duplicated observations in all columns except "Page":', df.drop(columns=['Page']).duplicated(keep='first').sum()/len(df)*100, '%')

Percentage of duplicated observations in all columns except "Page": 0.10928233240793854 %


Only 0.11% of rows are duplicated cases.

As stated before, this is probably because these cases changed places with others in different pages when a new query for the same year/month/tribunal was executed.

### Missing values:

In [20]:
for col in df.columns:
    print('Checking missing values in column', col, ':', (df['Corte'].isna() | df['Corte']=='').sum())

Checking missing values in column Competencia : 0
Checking missing values in column Corte : 0
Checking missing values in column Tribunal : 0
Checking missing values in column Page : 0
Checking missing values in column Rol : 0
Checking missing values in column Fecha : 0
Checking missing values in column Caratulado : 0


None of the columns have missing values.

### Tabulations:

In [21]:
df['Competencia'].value_counts(dropna=False)

Civil    2130262
Name: Competencia, dtype: int64

In [22]:
df['Corte'].value_counts(dropna=False)

C.A. de Santiago    2130262
Name: Corte, dtype: int64

Columns `Competencia` and `Corte` have only one value as expected, and no missings.

## Plots

In [23]:
def collapse(df, col):
    
    new_df = df[col].value_counts(dropna=False).reset_index().\
             rename(columns={'index': col, col: 'Cases'})

    return new_df

### Number of cases by Tribunal

In [24]:
df_temp = collapse(df, 'Tribunal')

alt.Chart(df_temp).mark_bar().encode(
    x = 'Cases:Q',
    y = 'Tribunal:N'
).properties(
    title = 'Number of Cases by Tribunal'
)

alt.Chart(...)

All Tribunals have around 70,000 cases, except `Juzgado de Letras de Colina`.

In [25]:
def horizontal_bar_chart(df, col, font_size=12, width=100):
    
    df_temp = collapse(df, col)
    
    bars = alt.Chart(df_temp).mark_bar().encode(
        y = 'Cases:Q',
        x = col+':O'
    ).properties(
        width = width
    )
    text = bars.mark_text(
        align='center',
        dy=-8,
        fontSize = font_size
    ).encode(
        text='Cases:Q'
    )

    return bars+text

### Number of cases by Year

In [26]:
df['Year'] = df['Fecha'].apply(lambda x: x[-4:])

In [27]:
horizontal_bar_chart(df, 'Year', width=180).properties(
    title = 'Number of Cases by Year'
)

alt.LayerChart(...)

2017 had an increase in the number of cases.

Other than the year from column `Fecha`, there is also a year in `Rol` column. We'll name this `Year2`.

In [28]:
df['Year2'] = df['Rol'].apply(lambda x: x[-4:])

In [29]:
horizontal_bar_chart(df, 'Year2', font_size=11, width=700).properties(
    title = 'Number of Cases by Year (from "Rol")'
)

alt.LayerChart(...)

The great majority of cases come from 2016 and 2017, though a few of them have a year from decades before. Perhaps the year in `Rol` is the year when the case began whereas the year in `Fecha` is when the final result was delivered?

In [30]:
print('Number of cases with year (from "Rol") before 2016:', len(df[df['Year2'].astype('int')<2016]))

Number of cases with year (from "Rol") before 2016: 250


In [31]:
print('Proportion of cases with year (from "Rol") before 2016:', len(df[df['Year2'].astype('int')<2016])/len(df))

Proportion of cases with year (from "Rol") before 2016: 0.00011735645662364535


### Number of cases by month

In [32]:
df['Month'] = df['Fecha'].apply(lambda x: int(x.split('/')[1]))

In [33]:
horizontal_bar_chart(df, 'Month', font_size=11, width=700).properties(
    title = 'Number of Cases by Month'
)

alt.LayerChart(...)

In [34]:
df['day'] = df['Fecha'].apply(lambda x: int(x.split('/')[0]))

### Number of cases by day

In [35]:
horizontal_bar_chart(df, 'day', font_size=8.5, width=870).properties(
    title = 'Number of Cases by day'
)

alt.LayerChart(...)

## Exporting the dataframe with the total cases

In [36]:
df.to_csv('master_data_table_appended.csv', index=False)